In [26]:
import os 
import math 
import pathlib 
import cv2
import shutil 
import numpy as np 
import pandas as pd 
import pickle 
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf 
import tensorflow.keras as keras 
from tensorflow.keras import layers 
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from sklearn.metrics import * 
from tensorflow.keras.layers import Dense, Input, Conv2D, BatchNormalization, Dropout, \
        LayerNormalization, GlobalAveragePooling2D, Lambda, Add, MaxPooling2D, ReLU, AveragePooling2D, LeakyReLU, Flatten

In [9]:
LAMBDA_u = 1  # ratio between the supervised loss and unsupervised loss in total loss
TAU = 0.9  # threshold
MU = 7    # unlabelled data ratio.
DECAY_RATE = 0.999
MOMENTUM = 0.9
IMG_WIDTH = 32 
IMG_HEIGHT = 32 
N_CHANNELS = 3 

In [12]:
class DataLoader: 
    """
        Class, will be useful for creating the BYOL dataset or dataset for the DownStream task 
            like classification or segmentation.
        Methods:
            __normalize(scope: private)
            __preprocess_img(scope: private)
             __get_valdata(scope: private)
            get_vit_dataset(scope: public)
        
        Property:
            dname(dtype: str)        : dataset name(supports cifar10, cifar100).
            augmentor(type)          : byol augmentor instance/object.
            nval(type: int)          : Number of validation data needed, this will be created by splitting the testing
                                       data.
            resize_shape(dtype: int) : Resize shape, bcoz pretrained models, might have a different required shape.
            normalize(dtype: bool)   : bool value, whether to normalize the data or not. 
    """
    
    def __init__(self, dname="cifar10", augmentor=None, unlabel_ratio=0.90,
                                             resize_shape=32, normalize=True, mu=10, label_data_bs=32): 
        assert dname in ["cifar10", 'cifar100'], "dname should be either cifar10 or cifar100"
        assert unlabel_ratio <= 0.99, "ValueError: nval value should be <= 0.99"
        
        __train_data, __test_data = self.__download_data(dname)
        self.__train_X, self.__train_y = __train_data
        self.__dtest_X, self.__dtest_y = __test_data 
        self.class_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                                           'dog', 'frog', 'horse', 'sheep', 'truck']
        self.augmentor = augmentor
        self.resize_shape = resize_shape
        
        self.__normalize() if normalize else None
        self.min_obj_cov_value = 0.7
        self.color_jitter_value = 0.1
        self.mu = mu 
        self.__tot_num_samples = len(self.__train_X)
        self.label_data_bs = label_data_bs
        self.__get_unlabeled_data(unlabel_ratio)
        
    def __len__(self): 
        return self.__train_X.shape[0] + self.__dtest_X.shape[0]
    
    def __repr__(self): 
        return f"Training Samples: {self.__train_X.shape[0]}, Testing Samples: {self.__dtest_X.shape[0]}"
    
    def __download_data(self, dname):
        """
            Downloads the data from the tensorflow website using the tensorflw.keras.load_data() method.
            Params:
                dname(type: Str): dataset name, it just supports two dataset cifar10 or cifar100
            Return(type(np.ndarray, np.ndarray))
                returns the training data and testing data
        """
        if dname == "cifar10": 
            train_data, test_data = tf.keras.datasets.cifar10.load_data()
        if dname == "cifar100": 
            train_data, test_data = tf.keras.datasets.cifar100.load_data()
            
        return train_data, test_data
    
    def __normalize(self): 
        """
            this method, will used to normalize the inputs.
        """
        self.__train_X = self.__train_X / 255.0
        self.__dtest_X = self.__dtest_X / 255.0
    
    def __label_preprocess(self, image, label): 
        """
            this method, will be used by the get_byol_dataset methos, which does a convertion of 
            numpy data to tensorflow data.
            Params:
                image(type: np.ndarray): image data.
            Returns(type; (np.ndarray, np.ndarray))
                returns the two different augmented views of same image.
        """
        try: 
            image = tf.image.convert_image_dtype(image, tf.float32)
            preprocessed_image = self.augmentor.weak_augment(image)
            
            return preprocessed_image, label
        
        except Exception as err:
            return err
        
    def __unlabel_preprocess(self, image): 
        """
            this method, will be used by the get_byol_dataset methos, which does a convertion of 
            numpy data to tensorflow data.
            Params:
                image(type: np.ndarray): image data.
            Returns(type; (np.ndarray, np.ndarray))
                returns the two different augmented views of same image.
        """
        try: 
            image = tf.image.convert_image_dtype(image, tf.float32)
            weak_augmented_img = self.augmentor.weak_augment(image)
            strong_augmented_img = self.augmentor.strong_augment(image)
            
            return (weak_augmented_img, strong_augmented_img)
        
        except Exception as err:
            return err
    
    def get_labeled_dataset(self, dataset_type="train"):
        """
            this method, will gives the byol dataset, which is nothing but a tf.data.Dataset object.
            Params:
                batch_size(dtype: int)    : Batch Size.
                dataset_type(dtype: str)  : which type of dataset needed, (train, test or val)
                
            return(type: tf.data.Dataset)
                returns the tf.data.Dataset for intended dataset_type, by preprocessing and converting 
                the np data.
        """
        try:
            if dataset_type == "train":
                tensorflow_data = tf.data.Dataset.from_tensor_slices((self.__labeled_X, self.__labeled_y))
                tensorflow_data = (
                tensorflow_data
                  #  .map(self.__label_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    .shuffle(1024)
                    .batch(self.label_data_bs, drop_remainder=True)
                    .prefetch(tf.data.experimental.AUTOTUNE)
                )
                return tensorflow_data  
            
            if dataset_type == "test":
                tensorflow_data = tf.data.Dataset.from_tensor_slices((self.__dtest_X, self.__dtest_y))
                tensorflow_data = (
                tensorflow_data
                    .shuffle(1024)
                    .batch(self.label_data_bs, drop_remainder=True)
                    .prefetch(tf.data.experimental.AUTOTUNE)
                )
                return tensorflow_data  
        
        except Exception as err:
            return err
        
    def get_unlabeled_dataset(self):
        """
            this method, will gives the byol dataset, which is nothing but a tf.data.Dataset object.
            Params:
                batch_size(dtype: int)    : Batch Size.
                dataset_type(dtype: str)  : which type of dataset needed, (train, test or val)
                
            return(type: tf.data.Dataset)
                returns the tf.data.Dataset for intended dataset_type, by preprocessing and converting 
                the np data.
        """
        try:
            unlabeled_bs = self.mu * self.label_data_bs
            tensorflow_data = tf.data.Dataset.from_tensor_slices(self.__unlabeled_X)
            tensorflow_data = (
                tensorflow_data
                .map(self.__unlabel_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .shuffle(1024)
                .batch(unlabeled_bs, drop_remainder=True)
                .prefetch(tf.data.experimental.AUTOTUNE)
            )
            return tensorflow_data  
        
        except Exception as err:
            return err
    
    def __get_unlabeled_data(self, unlabel_ratio):
        """
            this method is used to create a labeled and unlabeled data with the unlabeled_ratio..
            Params:
                nval(dtype: Int); Number of validation data needed, rest of test_X.shape[0] - nval, will be 
                                  testing data size.
            returns(type; np.ndarray, np.ndarray):
                returns the testing and validation dataset.
        """
        try: 
            n_unlabel_data = int(unlabel_ratio * self.__tot_num_samples)
            n_labeled_data = self.__tot_num_samples - n_unlabel_data
            
            ind_arr = np.arange(self.__tot_num_samples)
            
            labeled_inds = np.random.choice(ind_arr, n_labeled_data)
            unlabeled_inds = [i for i in ind_arr if not i in labeled_inds]
            
            self.labeled_inds = labeled_inds 
            self.unlabeled_inds = unlabeled_inds
            
            self.__labeled_X, self.__labeled_y = self.__train_X[labeled_inds], self.__train_y[labeled_inds]
            self.__unlabeled_X = self.__train_X[unlabeled_inds]
        except Exception as err:
            raise err

In [13]:
class FixMatchAugmentor: 
    """
        This class is used for the data augmentation for the byol model.
        Methods: 
            __random_flip(scope: private)
            __random_zoom(scope: private)
            augment(scope: public)
    """
    def __init__(self): 
        pass 
    
    def __random_horizontal_flip(self, image):
        """
            this method does a random crop with height and width of the crop are sampled randomly. it does the 
            crop with the height and width, then it does a resizing again to the original shape. And also it does
            a flip of the image
            Params:
                image(type: tf.Tensor)   : image data of type tensor.
                resize_shape(type: int)  : Size of the image.
            Return(type: tf.Tensor)
                returns the crop and resized image.
        """
        try:    
            image = tf.image.random_flip_left_right(image) 
            return image
        
        except Exception as err:
            return err
    
    @tf.function
    def __random_zoom(self, image):
        """
            this method, will do the color disortion augmentation for the given image.
            Params:
                image(type: tf.Tensor)   : image data of type tensor.
            Return(type: tf.Tensor)
                returns the crop and resized image.
        """
        try: 
            image = tf.keras.preprocessing.image.random_zoom(image, (0.5, 0.5), row_axis=0, col_axis=1, channel_axis=2)
            image = tf.keras.preprocessing.image.random_rotation(image, 20, row_axis=0, col_axis=1, channel_axis=2)
            return tf.convert_to_tensor(image)
        
        except Exception as error:
            return error
    
    @tf.function
    def weak_augment(self, image): 
        """
            this method will include all the augmentation as a pipeline(random crop, random flip, resize, and 
            color disortion), this augment method will be used by DataLoader class.
            Params:
                image(type: tf.Tensor)   : image data of type tensor.
                resize_shape(type: int)  : Size of the image.
            Return(type: tf.Tensor)
                returns the crop and resized image.
                
        """
        try: 
            
            image = self.__random_horizontal_flip(image)
            image = tf.image.random_brightness(image, max_delta=0.2)
            image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
            return image
        
        except Exception as error:
            print(error, error)
            return error
        
    #@tf.function
    def strong_augment(self, x, seed=(1, 2), rand_jpeg_quality=False, pad=4): 
        """
            this method will include all the augmentation as a pipeline(random crop, random flip, resize, and 
            color disortion), this augment method will be used by DataLoader class.
            Params:
                image(type: tf.Tensor)   : image data of type tensor.
                resize_shape(type: int)  : Size of the image.
            Return(type: tf.Tensor)
                returns the crop and resized image.
                
        """
        try: 

            x = tf.image.random_flip_left_right(x)
            x = tf.image.random_brightness(x, max_delta=0.9)
            x = tf.image.random_contrast(x, lower=0.2, upper=1.8)

            x = tf.image.random_hue(x, max_delta=0.2)
            x = tf.image.random_saturation(x, lower=0.2, upper=2)
          #  x = tf.image.random_crop(x, (16, 16, 3))
           # x = tf.image.resize(x, (32, 32))

            if rand_jpeg_quality:
                x = tf.image.random_jpeg_quality(x, min_jpeg_quality=50, max_jpeg_quality=100)

        #    x = tf.pad(x, paddings=[(0, 0), (pad, pad), (pad, pad), (0, 0)], mode="REFLECT")
          #  x = tf.image.stateless_random_crop(x, size=(32, width, 3), seed=seed)

            return x
        except Exception as error:
            print(error, error)
            return error

In [14]:
fixmatch_data_augmentor = FixMatchAugmentor()
fixmatch_dataloader = DataLoader("cifar10", fixmatch_data_augmentor, 0.9, 32, True, MU, 32)

In [15]:
labeled_ds = fixmatch_dataloader.get_labeled_dataset()
unlabeled_ds = fixmatch_dataloader.get_unlabeled_dataset()
test_ds = fixmatch_dataloader.get_labeled_dataset("test")

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [16]:
import tensorflow as tf


class Residual3x3Unit(tf.keras.layers.Layer):
    def __init__(self, channels_in, channels_out, stride, droprate=0., activate_before_residual=False):
        super(Residual3x3Unit, self).__init__()
        self.bn_0 = BatchNormalization(momentum=0.999)
        self.relu_0 = LeakyReLU(alpha=0.1)
        self.conv_0 =Conv2D(channels_out, kernel_size=3, strides=stride, padding='same', use_bias=False)
        self.bn_1 = BatchNormalization(momentum=0.999)
        self.relu_1 = LeakyReLU(alpha=0.1)
        self.conv_1 = Conv2D(channels_out, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.downsample = channels_in != channels_out
        self.shortcut = Conv2D(channels_out, kernel_size=1, strides=stride, use_bias=False)
        self.activate_before_residual = activate_before_residual
        self.dropout = Dropout(rate=droprate)
        self.droprate = droprate

    @tf.function
    def call(self, x, training=True):
        if self.downsample and self.activate_before_residual:
            x = self.relu_0(self.bn_0(x, training=training))
        elif not self.downsample:
            out = self.relu_0(self.bn_0(x, training=training))
        out = self.relu_1(self.bn_1(self.conv_0(x if self.downsample else out), training=training))
        if self.droprate > 0.:
            out = self.dropout(out)
        out = self.conv_1(out)
        return out + (self.shortcut(x) if self.downsample else x)


class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_units, channels_in, channels_out, unit, stride, droprate=0., activate_before_residual=False):
        super(ResidualBlock, self).__init__()
        self.units = self._build_unit(n_units, unit, channels_in, channels_out, stride, droprate, activate_before_residual)

    def _build_unit(self, n_units, unit, channels_in, channels_out, stride, droprate, activate_before_residual):
        units = []
        for i in range(n_units):
            units.append(unit(channels_in if i == 0 else channels_out, 
                        channels_out, stride if i == 0 else 1, droprate, activate_before_residual))
        return units

    @tf.function
    def call(self, x, training=True):
        for unit in self.units:
            x = unit(x, training=training)
        return x


class WideResNet(tf.keras.Model):
    def __init__(self, num_classes, depth=28, width=2, droprate=0., input_shape=(None, 32, 32, 3), **kwargs):
        super(WideResNet, self).__init__(input_shape, **kwargs)
        assert (depth - 4) % 6 == 0
        N = int((depth - 4) / 6)
        channels = [16, 16 * width, 32 * width, 64 * width]

        self.conv_0 = tf.keras.layers.Conv2D(channels[0], kernel_size=3, strides=1, padding='same', use_bias=False)
        self.block_0 = ResidualBlock(N, channels[0], channels[1], Residual3x3Unit, 1, droprate, True)
        self.block_1 = ResidualBlock(N, channels[1], channels[2], Residual3x3Unit, 2, droprate)
        self.block_2 = ResidualBlock(N, channels[2], channels[3], Residual3x3Unit, 2, droprate)
        self.bn_0 = BatchNormalization(momentum=0.999)
        self.relu_0 = LeakyReLU(alpha=0.1)
        self.avg_pool = AveragePooling2D((8, 8), (1, 1))
        self.flatten = layers.Flatten()
        self.dense = Dense(num_classes, activation="softmax")

    @tf.function
    def call(self, inputs, training=True):
        x = inputs
        x = self.conv_0(x)
        x = self.block_0(x, training=training)
        x = self.block_1(x, training=training)
        x = self.block_2(x, training=training)
        x = self.relu_0(self.bn_0(x, training=training))
        x = self.avg_pool(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

In [22]:
def create_model(image_size=None):
    model = Sequential()
  # Add the layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model
     

In [17]:
wide_resnet = WideResNet(10)

In [18]:
wide_resnet.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics="acc")

In [19]:
def create_pseudo_labels(weak_augmentation_predictions, threshold): 
    max_indexes = tf.math.argmax(weak_augmentation_predictions, axis=1)
    confidence = tf.reduce_max(weak_augmentation_predictions, axis=1)
    pseudo_labels = tf.where(confidence > threshold, max_indexes, -1)
    
    return pseudo_labels

def train_step(model,
               labeled_batch,
               unlabeled_batch, 
               optimizer,
               loss_function,
               metric_function,
               lambda_u,
               tau):
    """
        this method does the manual tr, ain step using the gradient tape.
        Params: 
            x(type: tf.data.Dataset): Independent Variable of type tf.data.Dataset.
            y(type: tf.data.Dataset): dependent Variable of type tf.data.Dataset.
            model(type; tf.models.Model): keras model, that will be used for the prediction.
            optimizer(tf.optimizer): Optimizer for updating the gradients.
            loss_fn(type: tf.losses): Loss function for calculating the loss of the model.
            train_acc_metric(type; tf.accuracy): Metric function for evaluation the data.
            
        Return(type: float)
            this method returns the loss value of the training data.
    """
    labeled_weak_augmentation_X, labeled_weak_augmentation_y = labeled_batch[0], labeled_batch[1]
    unlabeled_weak_augmentation_data = unlabeled_batch[0]
    unlabeled_strong_augmentation_data = unlabeled_batch[1]
  #  print(labeled_weak_augmentation_X, labeled_weak_augmentation_y)
   # print(unlabeled_weak_augmentation_data)
    
    num_pseudo_labels = 0
    
    with tf.GradientTape() as tape:
        # labeled loss
        y_pred = model(labeled_weak_augmentation_X)
        labeled_loss = loss_function(labeled_weak_augmentation_y, y_pred)
        
        # weak augmentation prediction and pseduolabels
        weak_augmentation_predictions = model(unlabeled_weak_augmentation_data)
        pseudo_labels = create_pseudo_labels(weak_augmentation_predictions, tau)
        
        # strong augmentation
        strong_augmentation_predictions = model(unlabeled_strong_augmentation_data)
        filtered_pseudo_labels = tf.boolean_mask(pseudo_labels, pseudo_labels != -1)
        filtered_y_pred_strong_augmentation = tf.boolean_mask(strong_augmentation_predictions, pseudo_labels != -1)
        
        if filtered_pseudo_labels.shape[0] > 0:
            unlabeled_loss = loss_function(filtered_pseudo_labels, filtered_y_pred_strong_augmentation)
            num_pseudo_labels += filtered_pseudo_labels.shape[0]
        else:
            unlabeled_loss = 0
            
        total_loss = labeled_loss + lambda_u * unlabeled_loss
    
    params = model.trainable_weights
    grads = tape.gradient(total_loss, params)
    optimizer.apply_gradients(zip(grads, params))
    
    return total_loss, labeled_loss, unlabeled_loss, num_pseudo_labels

In [23]:
import tqdm 
def manual_train(
                labeled_ds,
                unlabeled_ds,    
                model,
                optimizer,
                loss_fn,
                train_acc_metric,
                epochs=10, 
            ):

        """
            this method, does the manual train and test step for multiple epochs
            Params: 
                epochs(type; int): Number of epoch model needed to be trained.
                log_step(type:int): Number of train step.
                val_log_step(type:int): Number of val step.
                x(type: tf.data.Dataset): Independent Variable of type tf.data.Dataset.
                y(type: tf.data.Dataset): dependent Variable of type tf.data.Dataset.
                model(type; tf.models.Model): keras model, that will be used for the prediction.
                loss_fn(type: tf.losses): Loss function for calculating the loss of the model.
                val_acc_metric(type; tf.accuracy): Metric function for evaluation the data.

        """
        tot_loss = []   
        pseudo_labs = []
        lab_losses = []
        unlab_losses = []
        tot_num_pseudo_lab = 0
        tot_lab_loss = 0
        tot_unlab_loss = 0
        for epoch in range(epochs):
            print("\nStart of epoch %d" % (epoch,))

            # Iterate over the batches of the dataset
            for step, labeled_batch in tqdm.tqdm(enumerate(labeled_ds), total=len(labeled_ds)):
                #print(len(labeled_ds))
                for s, unlabeled_batch in enumerate(unlabeled_ds): 
                    loss_value, lab_loss, unlab_loss, num_pseudo_lab = train_step(model,
                                                                                labeled_batch,
                                                                                unlabeled_batch, 
                                                                                optimizer,
                                                                                loss_fn,
                                                                                train_acc_metric,
                                                                                LAMBDA_u,
                                                                                TAU)
                    tot_loss.append(float(loss_value))
                    tot_num_pseudo_lab += num_pseudo_lab
                    tot_lab_loss += lab_loss.numpy()
                    tot_unlab_loss += unlab_loss
                    break

            # Run a validation loop at the end of each epoch
            m_tot_loss = np.mean(tot_loss)
            m_lab_loss = tot_lab_loss/len(tot_loss)
            m_unlab_loss = tot_unlab_loss/len(tot_loss)
            pseudo_labs.append(tot_num_pseudo_lab)
            lab_losses.append(m_lab_loss)
            unlab_losses.append(m_unlab_loss)
            print(f"tot_loss: {m_tot_loss} lab_loss: {m_lab_loss} unlab_loss: {m_unlab_loss} Pseudo_labs: {tot_num_pseudo_lab}")
            
            if epoch % 5 == 0:
                model.save("checkpoint_model")
        return model, tot_loss, lab_losses, unlab_losses, pseudo_labs


In [28]:
decay_steps = 1000
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.1, decay_steps=decay_steps)
opt = tf.keras.optimizers.SGD(learning_rate=lr_decayed_fn, nesterov=True, momentum=0.9)

opt = keras.optimizers.Adam()
wideresnet = WideResNet(10)
model = create_model()
loss_func = keras.losses.SparseCategoricalCrossentropy()

trained_model, train_oss, lab_loss, unlab_loss, n_pseudo_labs = manual_train(labeled_ds, unlabeled_ds, model,
                                                                                         opt, loss_func, None, 120)


Start of epoch 0


100%|██████████| 156/156 [00:44<00:00,  3.50it/s]


tot_loss: 2.4031082407022133 lab_loss: 2.1810331092430997 unlab_loss: 0.22207508981227875 Pseudo_labs: 2846

Start of epoch 1


100%|██████████| 156/156 [00:45<00:00,  3.40it/s]


tot_loss: 2.160464597818179 lab_loss: 1.885596161469435 unlab_loss: 0.274868369102478 Pseudo_labs: 7290

Start of epoch 2


...

tot_loss: 0.026137864134328153 lab_loss: 0.00858203160219499 unlab_loss: 0.017555834725499153 Pseudo_labs: 34661

Start of epoch 118


100%|██████████| 156/156 [00:45<00:00,  3.41it/s]


tot_loss: 0.007743018692007097 lab_loss: 0.001537925781951216 unlab_loss: 0.006205091718584299 Pseudo_labs: 34846

Start of epoch 119


100%|██████████| 156/156 [00:44<00:00,  3.47it/s]

tot_loss: 0.003862665744143156 lab_loss: 0.000286907625776277 unlab_loss: 0.003575758310034871 Pseudo_labs: 34913


In [ ]:
with open("total_loss.obj", "wb") as f:
    pickle.dump(train_oss, f)
    
with open("label_loss.obj", "wb") as f:
    pickle.dump(lab_loss, f)
    
with open("unlabel_loss.obj", "wb") as f:
    pickle.dump(unlab_loss, f)
    
with open("n_pseudo_labels.obj", "wb") as f:
    pickle.dump(n_pseudo_labs, f)

In [29]:
trained_model.save("trained_model_1") 

In [30]:
(_, _), (test_X, test_y) = tf.keras.datasets.cifar10.load_data()

In [31]:
trained_model = keras.models.load_model('checkpoint_model')
test_X = test_X.astype("float")
pr_y = trained_model.predict(test_X)

313/313 [==============================] - 1s 2ms/step


In [32]:
pred_y = np.argmax(pr_y, axis=1)

In [36]:
accuracy_score(pred_y, test_y)

0.5046